# Alignment Error Visualization

This notebook collects COM data from the database and tries to quantify some alignment errors. The main results are shown in the plots at the end of the notebook.

In [1]:
import os
import sys
from pathlib import Path

import numpy as np
import pandas as pd
from collections import OrderedDict
from scipy.ndimage import affine_transform
from skimage import measure
import SimpleITK as sitk
import matplotlib.pyplot as plt


PIPELINE_ROOT = Path('./').absolute().parents[1]
PIPELINE_ROOT = PIPELINE_ROOT.as_posix()
sys.path.append(PIPELINE_ROOT)
print(PIPELINE_ROOT)


/home/eddyod/programming/pipeline/src


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from library.controller.sql_controller import SqlController
from library.image_manipulation.filelocation_manager import FileLocationManager
from library.atlas.atlas_utilities import affine_transform_point, get_affine_transformation, \
fetch_coms, list_coms, compute_affine_transformation, affine_transform_volume
from library.atlas.brain_structure_manager import BrainStructureManager
from library.utilities.utilities_process import M_UM_SCALE, SCALING_FACTOR, random_string, \
read_image, write_image


In [4]:
def sum_square_com(com):
    ss = np.sqrt(sum([s**2 for s in com]))
    return ss

def apply_affine_transformation(volume, matrix):
    """Apply an affine transformation to a 3D volume."""
    transformed_volume = affine_transform(volume, matrix, offset=0, order=1)
    return transformed_volume

def load_ants_affine_mat(mat_file_path):
    # Load .mat file

    rotation = np.array([
    [0.941498875617981, 0.19202075898647308, -0.08786671608686447], 
    [-0.198630228638649, 0.781607985496521, 0.015325229614973068],
    [0.06243732571601868, 0.0008880351670086384, 0.8622015118598938]
    ]
    )
    translation = np.array([266.45843505859375, 150.64747619628906, -138.71780395507812])
    center = np.array([730.082275390625, 373.0629577636719, 569.5])
    # Apply rotation around center
    affine = np.eye(4)
    affine[:3, :3] = rotation
    affine[:3, 3] = translation + center - rotation @ center

    return affine

def convert_lps_to_sar(affine_lps):
    # Flip L (x) and P (y) axes: LPS to RAS => flip x and y
    flip = np.diag([-1, -1, 1, 1])
    affine_sar = flip @ affine_lps @ flip
    return affine_sar

In [18]:
moving_name = 'MD594'
fixed_name = 'Allen'
moving_all = list_coms(moving_name, scaling_factor=10)
fixed_all = list_coms(fixed_name, scaling_factor=10)
common_keys = list(moving_all.keys() & fixed_all.keys())
bad_keys = ('RtTg', 'AP')
#bad_keys = ('RtTg',)
#bad_keys = ()
good_keys = set(common_keys) - set(bad_keys)

moving_src = np.array([moving_all[s] for s in good_keys])
fixed_src = np.array([fixed_all[s] for s in good_keys])
print(len(common_keys))

37


In [27]:
from scipy.io import loadmat
reg_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
transform_file = 'MD594_Allen_10.0x10.0x10.0um_Affine_inverse.mat'
transform_path = os.path.join(reg_path, moving_name, transform_file)
transform = sitk.ReadTransform(transform_path)
transform = sitk.AffineTransform(transform)
print(transform)

itk::simple::AffineTransform
 AffineTransform (0x5d1bd46d15e0)
   RTTI typeinfo:   itk::AffineTransform<double, 3u>
   Reference Count: 1
   Modified Time: 703
   Debug: Off
   Object Name: 
   Observers: 
     none
   Matrix: 
     0.986549 -0.218002 0.103984 
     0.229206 1.17622 0.00316312 
     -0.0745162 0.00826388 1.14064 
   Offset: [-182.23, -475.044, 148.324]
   Center: [1041.58, 539.235, 443.876]
   Translation: [-267.639, -139.88, 137.591]
   Inverse: 
     0.965242 0.179521 -0.0884922 
     -0.188267 0.815184 0.0149024 
     0.0644218 0.0058219 0.870814 
   Singular: 0



In [19]:
#transformation_matrix = get_affine_transformation(moving_name=moving_name, fixed_name=fixed_name, 
#                                                  scaling_factor=1)
transformation_matrix = compute_affine_transformation(moving_src, fixed_src)
print(transformation_matrix)

[[ 8.98433334e-01 -2.65091877e-01  5.95423255e-02 -2.13381393e+01]
 [ 3.65894141e-01  1.13223847e+00  4.76580505e-02 -6.40295926e+02]
 [-5.71300691e-02 -1.34311696e-02  1.17741204e+00  2.81177915e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [20]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    moving0 = np.array(moving_all[structure])
    fixed0 = np.array(fixed_all[structure]) 
    transformed = affine_transform_point(moving0, transformation_matrix)
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))
# MD589 to Allen RMS 260.0211852431133
# MD585 to Allen RMS 263.314352291951
# MD594 to Allen RMS 250.79820210419254
# AtlasV8 DB to Allen RMS 238.5831606646421
# MD585 to MD589 RMS 18.2658167690059

RMS 25.07982021040849


In [37]:
transformed_elastix = {'10N_L': (1245.825565, 583.011918, 541.248353), '10N_R': (1241.078327, 575.816747, 604.932277), '12N': (1234.494263, 599.28526, 571.130307), '3N_L': (916.08634, 393.30332, 556.006873), '3N_R': (915.61113, 395.207295, 587.248926), '4N_L': (954.697061, 397.285593, 544.243496), '4N_R': (955.039712, 398.093118, 598.067921), '5N_L': (1018.137495, 520.533489, 409.881654), '5N_R': (1019.341376, 525.263033, 727.129661), '6N_L': (1068.668552, 527.691811, 539.596871), '6N_R': (1073.775168, 536.06989, 613.966415), '7N_L': (1083.300099, 645.703034, 437.587277), '7N_R': (1087.207846, 667.340715, 701.69945), '7n_L': (1045.956409, 566.507496, 447.340355), '7n_R': (1054.223742, 570.088494, 689.160729), 'AP': (1244.090532, 552.686906, 571.760582), 'Amb_L': (1160.638022, 636.809303, 427.495323), 'Amb_R': (1165.984467, 653.856579, 711.948937), 'DC_L': (1123.751951, 469.786863, 315.849668), 'DC_R': (1131.744401, 461.873771, 812.028033), 'IC': (1034.298923, 232.223501, 566.79765), 'LC_L': (1045.189532, 448.35284, 475.846568), 'LC_R': (1050.580737, 454.508383, 663.837542), 'LRt_L': (1233.211435, 702.321565, 443.053938), 'LRt_R': (1234.112904, 699.030951, 697.872942), 'PBG_L': (956.449566, 360.380351, 354.569236), 'PBG_R': (958.509989, 371.76958, 788.302734), 'Pn_L': (895.19021, 624.89548, 501.952816), 'Pn_R': (894.018076, 634.206201, 645.330084), 'RMC_L': (888.025505, 450.25689, 511.06366), 'RMC_R': (884.667685, 457.312469, 622.186793), 'RtTg': (940.155521, 592.96321, 566.352029), 'SC': (915.771574, 229.443045, 566.999621), 'SNC_L': (835.409636, 512.465283, 429.363112), 'SNC_R': (828.798088, 507.753302, 711.989699), 'SNR_L': (865.722722, 506.533869, 407.481066), 'SNR_R': (857.665458, 509.930588, 719.266855), 'Sp5C_L': (1301.972326, 627.651522, 413.166233), 'Sp5C_R': (1298.198967, 619.19027, 728.645978), 'Sp5I_L': (1222.916041, 590.080036, 364.984897), 'Sp5I_R': (1235.268962, 588.638673, 774.199137), 'Sp5O_L': (1141.028347, 577.329771, 364.832388), 'Sp5O_R': (1151.739201, 583.391376, 768.576876), 'Tz_L': (983.843401, 656.759861, 508.510555), 'Tz_R': (986.412422, 666.944614, 626.51371), 'VCA_L': (1048.046155, 509.507162, 297.514027), 'VCA_R': (1061.438954, 507.154848, 860.320689), 'VCP_L': (1095.838599, 548.173446, 299.199061), 'VCP_R': (1115.532484, 526.161194, 846.116132), 'VLL_L': (938.78711, 555.30814, 396.500515), 'VLL_R': (940.407198, 557.855505, 737.535395)}


In [38]:
df_list = []
error = []
transformed_dict = {}
for structure in common_keys:
    transformed = transformed_elastix[structure]
    fixed0 = np.array(fixed_all[structure]) 
    difference = [a - b for a, b in zip(transformed, fixed0)]
    ss = sum_square_com(difference)
    row = [structure, np.round(moving0), np.round(fixed0), 
           np.round(transformed), np.round(difference), ss]
    df_list.append(row)
    error.append(ss)
    transformed_dict[structure] = transformed
print('RMS', sum(error)/len(df_list))


RMS 24.81866454700675


In [31]:
#transformation_matrix = np.hstack([transformation_matrix, t])
#transformation_matrix = np.vstack([transformation_matrix, np.array([0, 0, 0, 1])])
#print(transformation_matrix)
structure = 'SC'
try:
    com = moving_all[structure]
except KeyError:
    structure = common_keys[0]
    com = moving_all[structure]
#com = [1095, 392, 519]
print(f'{moving_name} {structure} non trans {np.round(np.array(com))}')
transformed_structure = affine_transform_point(com, transformation_matrix)
x,y,z = com

print(f'{moving_name} {structure} apply ants {x=}, {y=} {z=}')

print(f'{moving_name} {structure} apply trans {np.round(transformed_structure/1)}')
print(f'{fixed_name} {structure} {np.round(np.array(fixed_all[structure]))}')
diff = transformed_structure - fixed_all[structure]
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')
elastix_com = np.array([915, 230, 568])
diff = elastix_com - np.array(fixed_all[structure])
print(f'{moving_name}->{fixed_name} {structure} {np.round(diff)}')

MD594 SC non trans [1119.  385.  517.]
MD594 SC apply ants x=1119.232944161371, y=384.50369985453443 z=516.5074759237448
MD594 SC apply trans [913. 229. 567.]
Allen SC [914. 239. 569.]
MD594->Allen SC [ -1. -10.  -2.]
MD594->Allen SC [ 1. -9. -1.]


In [39]:
columns = ['structure', moving_name, fixed_name, 'transformed', 'difference', 'sumsquares']
df = pd.DataFrame(df_list, columns=columns)
df.index.name = 'Index'
df = df.round(4)
df.sort_values(by=['sumsquares'], inplace=True)
#df.to_csv('/home/eddyod/programming/pipeline/docs/sphinx/source/_static/results.csv', index=False)
df.head(50)

,structure,MD594,Allen,transformed,difference,sumsquares
Index,,,,,,
17,SNC_L,"[1234.0, 664.0, 526.0]","[835.0, 512.0, 429.0]","[835.0, 512.0, 429.0]","[0.0, -0.0, 0.0]",0.0010
12,LRt_R,"[1234.0, 664.0, 526.0]","[1234.0, 699.0, 699.0]","[1234.0, 699.0, 698.0]","[-0.0, -0.0, -1.0]",1.4201
8,5N_R,"[1234.0, 664.0, 526.0]","[1019.0, 529.0, 729.0]","[1019.0, 525.0, 727.0]","[-0.0, -3.0, -2.0]",3.9298
24,LRt_L,"[1234.0, 664.0, 526.0]","[1234.0, 699.0, 439.0]","[1233.0, 702.0, 443.0]","[-1.0, 3.0, 4.0]",5.1425
13,IC,"[1234.0, 664.0, 526.0]","[1040.0, 232.0, 568.0]","[1034.0, 232.0, 567.0]","[-6.0, -0.0, -1.0]",5.7508
29,5N_L,"[1234.0, 664.0, 526.0]","[1019.0, 529.0, 409.0]","[1018.0, 521.0, 410.0]","[-1.0, -8.0, 1.0]",8.2361
25,SNC_R,"[1234.0, 664.0, 526.0]","[836.0, 512.0, 709.0]","[829.0, 508.0, 712.0]","[-7.0, -5.0, 3.0]",8.6027
19,SC,"[1234.0, 664.0, 526.0]","[914.0, 239.0, 569.0]","[916.0, 229.0, 567.0]","[2.0, -9.0, -2.0]",9.7765
1,7N_R,"[1234.0, 664.0, 526.0]","[1085.0, 678.0, 705.0]","[1087.0, 667.0, 702.0]","[2.0, -10.0, -3.0]",10.7512


In [ ]:
outpath = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data/DK55/com'
for structure, com in moving_all.items():
    comfile = structure + '.txt'
    compath = os.path.join(outpath, comfile)
    np.savetxt(compath, com)

In [ ]:
um = 50
registration_path = '/net/birdstore/Active_Atlas_Data/data_root/brains_info/registration'
base_com_path = '/net/birdstore/Active_Atlas_Data/data_root/atlas_data'
for brain in [moving_name, fixed_name]:
    brain_point_path = os.path.join(registration_path, brain, f'{brain}_{um}um_sagittal.pts')
    brain_com_path = os.path.join(base_com_path, brain, 'com')
    comfiles = sorted(os.listdir(brain_com_path))
    with open(brain_point_path, 'w') as f:
        f.write('point\n')
        f.write(f'{len(common_keys)}\n')
        for comfile in comfiles:
            structure = comfile.replace('.txt','')
            if structure in common_keys:
                #print(structure)
                compath = os.path.join(brain_com_path, comfile)
                x,y,z = np.loadtxt(compath)
                f.write(f'{round(x/um,4)} {round(y/um,4)} {round(z/um,4)}')
                f.write('\n')


In [ ]:
def ants_3d_to_scipy_2d(affine_3d, plane='axial', slice_index=0):
    """
    Convert a 3D ANTs affine transformation matrix to a 2D affine transform
    suitable for scipy.ndimage.affine_transform.

    Parameters:
        affine_3d (np.ndarray): A 4x4 affine matrix from ANTs.
        plane (str): Plane to slice through ('axial', 'coronal', 'sagittal').
        slice_index (int): Index of the slice in the chosen plane.

    Returns:
        matrix_2d (np.ndarray): 2x2 affine transformation matrix.
        offset_2d (np.ndarray): Length-2 offset vector.
    """
    if affine_3d.shape != (4, 4):
        raise ValueError("Expected a 4x4 affine transformation matrix.")

    # Extract rotation+scaling and translation components
    rotation_scaling = affine_3d[:3, :3]
    translation = affine_3d[:3, 3]

    if plane == 'axial':
        matrix_2d = rotation_scaling[:2, :2]
        offset_2d = translation[:2] + rotation_scaling[:2, 2] * slice_index
    elif plane == 'coronal':
        matrix_2d = rotation_scaling[[0,2], :][:, [0,2]]
        offset_2d = translation[[0,2]] + rotation_scaling[[0,2], 1] * slice_index
    elif plane == 'sagittal':
        matrix_2d = rotation_scaling[1:3, 1:3]
        offset_2d = translation[1:3] + rotation_scaling[1:3, 0] * slice_index
    else:
        raise ValueError("Plane must be 'axial', 'coronal', or 'sagittal'.")

    return matrix_2d, offset_2d

In [ ]:
from taskqueue import LocalTaskQueue
import igneous.task_creation as tc
from cloudvolume import CloudVolume
tq = LocalTaskQueue(parallel=2)

In [36]:
print(len(moving_all.keys()))
for k, (x,y,z) in moving_all.items():
    print(x,y,z)

51
1539.332121000432 563.1368328522692 514.9211076280042
1528.4149895025182 557.8558065202129 567.9256064690027
1529.9469782583617 578.9357763650071 539.9434349188266
1160.854644401185 513.8434280282418 511.22647657841134
1158.7016716143776 514.9733514334788 537.4240444299248
1201.3504870825234 505.4002771895151 502.8178137651822
1198.1934741845312 505.042172430262 547.9329983249581
1304.6775188479144 585.4881182191522 395.4370762079407
1285.244566273447 583.6025119440335 661.3224855079833
1348.1814081886816 573.5728659589793 506.02486402486403
1350.2606322710722 577.3931777201824 568.6962509563887
1399.0645525974026 663.8848948910006 423.9118826296338
1390.1790554133372 675.3908057635209 645.7886810162767
1341.3639186647847 612.6461406132563 428.8836617927527
1333.9123017125303 608.9476867119175 631.8094761984114
1527.9908313128724 539.2407671891922 539.6931899641577
1475.0774656729343 633.44287215194 417.9489105935387
1465.0996391961328 640.5523823423467 656.8045007032349
1404.446914